In [50]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image 
import numpy as np

class CoreImages():

    def __init__(self, core_images, core_x, core_y, slice):
        """
        core_images: a list of core image paths
        core_x: Tuple (x1,x2) of the x coordinates of the core trays
        core_y: Tuple of tuples ((y1,y2),(y1,y2),(y1,y2),...) with y coordinates for each of the cores within the image
        slice: width to slice each core image into. If not integer divisible by the length of the core, the end of the core will be discarded
        """
        
        self.core_images = core_images
        self.core_x = core_x
        self.core_y = core_y
        self.slice = slice
        assert self.slice > 0, "Slice should be greater than 0"
        self.cores_per_image = len(self.core_y)

        self.core_length = core_x[1] - core_x[0]
        self.slices_per_core = np.floor(self.core_length / slice)

    def __getitem__(self, idx):
        return self.core_images[idx]

    def slice_cores(self, dir):
        """slice each image into """
        self.core_slices = []
        left, right = self.core_x

        for core_image in self.core_images:
            for n_core in range(self.cores_per_image):
                img = Image.open(core_image)
                core = img.crop((left,self.core_y[n_core][0],right,self.core_y[n_core][1]))
                core.save(f"{dir}/{os.path.basename(core_image)}_{n_core}.jpg")


                
                img.close()
    

image_dir = "data/Core Images/3-03- 11/"
files = os.listdir(image_dir)
images = [os.path.join(image_dir, f) for f in files]


core_x = 730
core_x_size = 6240
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice = 624

cores = CoreImages(images, core_x, core_y, slice)

cores.slice_cores("data/")